In [1]:
import random
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import dbscan
from sklearn.cluster import k_means
from sklearn.cluster import KMeans
from tqdm import tqdm_notebook
from collections import Counter

plt.rcParams['font.family'] = 'NanumGothic'
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.options.display.float_format = '{:.3f}'.format

In [53]:
# 그럼 나는 플스 아디별 송 아디 이름을 구해
plst_song = pd.read_csv('플스아디곡아디.csv', encoding='utf-8')
plst_song['songs'] = plst_song['songs'].astype('str')
del plst_song['Unnamed: 0']
trans_id_idx = plst_song.set_index('id')['songs']
trans_id_idx 

id
61281     525514
61281     129701
61281     383374
61281     562083
61281     297861
           ...  
100389    111365
100389     51373
100389    640239
100389     13759
100389    154078
Name: songs, Length: 5285871, dtype: object

In [30]:
trans_id_idx.index.unique()

Int64Index([ 61281,  10532,  76951, 147456,  27616,  69252,  45339,  36557,
             70741,  10288,
            ...
             53627,   2531, 120713, 145872, 135687, 120325, 106976,  11343,
            131982, 100389],
           dtype='int64', name='id', length=115071)

In [56]:
item_tuple_to_cnt = {}
for i in tqdm_notebook(trans_id_idx.index.unique()): # 고유한 플레이리스트를 하나씩 뽑아냄. 그럼 해당 플스트에 속한 곡들이 나옴
    items_ = trans_id_idx.loc[i] # 그 플레이리스트에 속한 곡들을 저장
    if type(items_) is not str:
        visit = set()
        for j in range(len(items_)): # 그 플스트의 곡들 중 하나를 대상으로
            tmp = list(items_) # (곡들을 리스트로 만들고)
            item = tmp.pop(j) # (그 리스트에서 하나씩 튀어나오게 함)
            if item in visit: # 그 곡이 visit 집합 안에 있으면
                continue # 다시 첨으로
            visit.add(item)  # visit 집합 안에 포함되어 있지 않으면 추가함
            for k in set(tmp): # 그 플스트의 고유한 곡 내역에서 하나씩 뽑아냄
                item_tuple_to_cnt[(item,k)] = item_tuple_to_cnt.get((item,k), 0) + 1 # 둘이 같이 나온 수를 세는 것
                # 딕셔너리에 그 플스트의 곡들 중 하나랑, 그 플스트에 있는 곡 중 하나를 뽑아내서 key로 만들고,
                # 그 key에 대한 값을 return하고 1을 더함. 키에 대한 값이 없으면 0을 리턴하고 1을 더함
                # 그럼 다음 플스트에서 똑같은 곡이 같이 또 나오면 계속 1이 더해지면서 동시 출현 횟수를 구하는 것.

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [57]:
item_tuple_to_cnt # 두 아이템이 같이 나온 횟수

{('525514', '121455'): 1,
 ('525514', '531057'): 1,
 ('525514', '297861'): 1,
 ('525514', '461973'): 1,
 ('525514', '324992'): 1,
 ('525514', '660493'): 1,
 ('525514', '383374'): 1,
 ('525514', '129701'): 2,
 ('525514', '706183'): 1,
 ('525514', '223955'): 1,
 ('525514', '562083'): 1,
 ('525514', '127099'): 1,
 ('525514', '205238'): 1,
 ('525514', '72552'): 1,
 ('525514', '139541'): 1,
 ('525514', '50104'): 1,
 ('525514', '650298'): 1,
 ('525514', '351214'): 1,
 ('129701', '121455'): 1,
 ('129701', '531057'): 1,
 ('129701', '297861'): 2,
 ('129701', '461973'): 1,
 ('129701', '324992'): 1,
 ('129701', '660493'): 1,
 ('129701', '383374'): 1,
 ('129701', '706183'): 1,
 ('129701', '223955'): 1,
 ('129701', '562083'): 1,
 ('129701', '127099'): 1,
 ('129701', '72552'): 1,
 ('129701', '205238'): 1,
 ('129701', '525514'): 2,
 ('129701', '139541'): 1,
 ('129701', '50104'): 1,
 ('129701', '650298'): 1,
 ('129701', '351214'): 1,
 ('383374', '121455'): 1,
 ('383374', '531057'): 1,
 ('383374', '297

In [58]:
item_to_cnt = {}
all_cnt = 0
for i in tqdm_notebook(trans_id_idx.index.unique()): # 각 플스트 아이디에서
    items = trans_id_idx.loc[i] # 그 플스트에 수록된 곡들
    if type(items) is not str:
        all_cnt += 1
        for j in set(items): # 고유 곡들
            item_to_cnt[j] = item_to_cnt.get(j, 0) + 1
item_to_cnt # 전체 플레이리스트에서 총 수록된 횟수

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


{'121455': 1,
 '562083': 15,
 '531057': 40,
 '297861': 57,
 '525514': 7,
 '383374': 1,
 '127099': 1,
 '139541': 24,
 '351214': 2,
 '324992': 1,
 '461973': 7,
 '129701': 3,
 '205238': 3,
 '650298': 46,
 '660493': 4,
 '706183': 1,
 '223955': 1,
 '72552': 63,
 '50104': 3,
 '120377': 15,
 '204499': 30,
 '192882': 43,
 '692078': 18,
 '532114': 853,
 '586541': 417,
 '244000': 102,
 '75801': 26,
 '152422': 1207,
 '696302': 34,
 '33389': 204,
 '129258': 11,
 '97749': 2,
 '6546': 1103,
 '409869': 1,
 '103741': 4,
 '414721': 23,
 '571802': 48,
 '181101': 8,
 '548636': 84,
 '383960': 244,
 '315216': 24,
 '602724': 105,
 '413920': 8,
 '645653': 37,
 '631268': 2,
 '497066': 1111,
 '41749': 12,
 '389529': 311,
 '472144': 7,
 '173634': 77,
 '442765': 18,
 '244277': 38,
 '432406': 54,
 '37741': 29,
 '425946': 197,
 '395416': 390,
 '675945': 45,
 '61435': 20,
 '461062': 135,
 '117205': 3,
 '200183': 30,
 '471385': 99,
 '443513': 23,
 '686560': 20,
 '348801': 54,
 '666852': 101,
 '176874': 31,
 '617795'

In [60]:
import pickle
# load
with open('곡별테이블.pickle', 'rb') as f:
    items = pickle.load(f)
items = items[['song_id', 'song_name']]
items.head()

,song_id,song_name
0,0,Feelings
1,3,Feeling Right (Everything Is Nice) (Feat. Popc...
2,4,그남자 그여자
3,5,Para Los Enamorados
4,6,Sibelius : Valse Triste Op.44 (시벨리우스 : 슬픈 왈츠 작...


In [61]:
upgraded_dict = {}
for t in tqdm_notebook(item_tuple_to_cnt.keys()):
    if item_tuple_to_cnt[t] >= 5: # 5번 이상 다른 것과 함께 재생된 노래만
        upgraded_dict[t] = (item_tuple_to_cnt[t] / item_to_cnt[t[0]] / item_to_cnt[t[1]] * all_cnt , item_tuple_to_cnt[t])
        # x가 나올 때 y가 나올 확률: 신뢰도를 y의 총 등장 횟수(지지도)로 나눔

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [62]:
upgraded_dict

{('432406', '120377'): (710.1234567901234, 5),
 ('432406', '204499'): (426.074074074074, 6),
 ('432406', '192882'): (297.26098191214464, 6),
 ('432406', '692078'): (591.7695473251028, 5),
 ('432406', '75801'): (409.68660968660964, 5),
 ('432406', '129258'): (1162.0202020202019, 6),
 ('432406', '414721'): (463.12399355877614, 5),
 ('432406', '571802'): (221.91358024691357, 5),
 ('432406', '315216'): (532.5925925925926, 6),
 ('432406', '413920'): (1597.7777777777776, 6),
 ('432406', '645653'): (345.46546546546546, 6),
 ('432406', '497066'): (11.505150515051504, 6),
 ('432406', '41749'): (1065.1851851851852, 6),
 ('432406', '389529'): (41.10039299749911, 6),
 ('432406', '173634'): (138.3357383357383, 5),
 ('432406', '442765'): (591.7695473251028, 5),
 ('432406', '244277'): (280.31189083820664, 5),
 ('432406', '37741'): (367.3052362707535, 5),
 ('432406', '425946'): (54.07031396879112, 5),
 ('432406', '675945'): (236.70781893004113, 5),
 ('432406', '61435'): (639.1111111111111, 6),
 ('4324

In [63]:
items_ = items.copy()
items_ = items_.set_index('song_id')
items_

,song_name
song_id,
0,Feelings
3,Feeling Right (Everything Is Nice) (Feat. Popc...
4,그남자 그여자
5,Para Los Enamorados
6,Sibelius : Valse Triste Op.44 (시벨리우스 : 슬픈 왈츠 작...
...,...
707984,Coffin For Head Of State
707985,Change Of Heart
707986,스치듯 안녕


In [89]:
sorted(upgraded_dict.items(), key=lambda x : -x[1][0])[:10][0][0]

('612857', '432081')

In [112]:
items_[items_['song_name']=='밤편지'] # 144663이 아이유노래

,song_name
song_id,
144663,밤편지
214221,밤편지
239630,밤편지
241245,밤편지
254041,밤편지
425102,밤편지
467894,밤편지
527898,밤편지
569626,밤편지


In [116]:
def item_reco1(song_id):
    name = items_.loc[song_id, 'song_name']
    cnt = 0
    print('현재노래 : {}'.format(name))
    print('연관노래아이디, 연관노래제목, 연관지수')
    for i in sorted(upgraded_dict.items(), key=lambda x : -x[1][0]):
        if i[0][0] == str(song_id) and i[0][1] != str(song_id): # 똑같은 노래 두 개는 어쨌든 연관지수가 1이니.
            print('{}, {}, {:.2f}'.format(i[0][1],
                                           items_.loc[int(i[0][1]), 'song_name'],
                                           i[1][0]))
            cnt += 1
            if cnt == 10: # 10개만 추천해주는 것
                break
    if cnt == 0:
        print('5번 이상 동시 재생된 연관노래가 없습니다.')
# 연관지수==향상도(lift)

In [117]:
item_reco1(532114)

현재노래 : 요즘 너 말야
연관노래아이디, 연관노래제목, 연관지수
2840, 말할거에요, 134.87
460108, 사노라면, 112.39
318282, 힘든가요? (wiv J Rabbit Mix), 112.39
460376, 하늘을 봐, 101.15
238934, Where Do We Go Now, 101.15
143434, 자격, 101.15
558921, 나의 일, 101.15
215604, 떠나가줄래, 96.33
309352, Nice (Skinny Jeans, White T-shirt & Ponytail) (Feat. PRiVAT-E), 89.91
669717, 자전거 여행, 89.91


In [120]:
item_reco1(497066)

현재노래 : 기다리다
연관노래아이디, 연관노래제목, 연관지수
143608, 잡아줘요, 62.13
639280, Hero, 57.53
519579, 속마음, 55.76
229751, 꼬마 - I Cry, 55.01
675952, 내 곁에서 떠나가지 말아요 (원곡가수 빛과 소금), 53.84
382560, 외사랑, 47.07
165248, 서른 즈음에, 47.07
97911, 그댄 모를테지만, 47.07
126673, 단꿈, 44.38
204549, 키다리아저씨 (Acoustic Ver.), 43.14


In [122]:
item_reco1(33389)

현재노래 : 그런 사람 또 없습니다
연관노래아이디, 연관노래제목, 연관지수
566814, 운명의 끈, 563.92
30195, 다줄꺼야 (이승기), 469.93
124951, 길(조관우), 469.93
259626, 사랑을 믿나요, 451.14
187474, 십년이지나도 (Remix), 451.14
213001, 기억 속으로 (원곡가수 이은미), 451.14
575769, 믿나요, 417.72
521137, 이별도 사랑이다 (경수 역/김민종 테마), 402.80
584943, 사랑보다 깊은 상처, 400.20
508276, 이별길, 396.83


In [123]:
item_reco1(144663)

현재노래 : 밤편지
연관노래아이디, 연관노래제목, 연관지수
566741, Hey!, 52.89
363213, Schumann : 5 Stucke Im Volkston Op.102 - II. Langsam (슈만 : 5개의 민요풍의 소품 작품번호 102 - 2번), 48.82
603498, Voulez-Vous (From &#39;Mamma Mia!&#39; Original Motion Picture Soundtrack), 48.82
667564, When All Is Said And Done (From &#39;Mamma Mia!&#39; Original Motion Picture Soundtrack), 45.34
284376, Does Your Mother Know (From &#39;Mamma Mia!&#39; Original Motion Picture Soundtrack), 45.34
621320, 편의점 앞에서, 44.08
399623, 궁금해줘요 (Feat. 백선) (Drama Ver.), 42.31
340471, SOS (From &#39;Mamma Mia!&#39; Original Motion Picture Soundtrack), 42.31
60313, Take A Chance On Me (From &#39;Mamma Mia!&#39; Original Motion Picture Soundtrack), 42.31
638165, 네가 있으면 좋겠어, 41.35
